<a href="https://colab.research.google.com/github/smwongela/NLP_Transformers/blob/main/accelerate_mrpc_classification_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149.9 MB 27 kB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
  Attempting uninst

In [2]:
! pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-_ha3qpuq
  Running command git clone -q https://github.com/huggingface/accelerate /tmp/pip-req-build-_ha3qpuq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for accelerate: filename=accelerate-0.11.0.dev0-py3-none-any.whl size=116978 sha256=44039301837b84d29f3c41a5979b4eca0d1ad31c81b66cdf0150e923fd735d1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-p5gkhqx2/wheels/81/c1/23/6068c1115888b4dd7da88f966c002c30840985c047f6cc1653
Successfully built accelerate


In [3]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 212 kB 57.0 MB/s 
     |████████████████████████████████| 140 kB 40.2 MB/s 
     |████████████████████████████████| 127 kB 54.1 MB/s 
     |████████████████████████████████| 271 kB 46.0 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 52.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.315 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-clien

In [4]:
import torch
from torch.utils.data import DataLoader
from accelerate import Accelerator, DistributedType

In [5]:
from datasets import load_dataset,load_metric

In [6]:
from transformers import(
     AutoTokenizer, 
     DataCollatorWithPadding,
     AutoModelForSequenceClassification,
     get_linear_schedule_with_warmup,
     set_seed,
     )
from tqdm.auto import tqdm
import datasets
import transformers

In [7]:
##Load datasets
raw_datasets=load_dataset("glue","mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [9]:
raw_datasets['train'][0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(raw_datasets["train"])

,sentence1,sentence2,label,idx
0,"Shares of National were losing 95 cents , or 3.9 % , to $ 23.73 in afternoon New York Stock Exchange trading .","Walgreen 's shares were up 4.6 % , or $ 1.42 , to $ 32.11 in midday New York Stock Exchange trading .",not_equivalent,1280
1,"Dynes will get $ 395,000 a year , an increase over Atkinson 's current salary of $ 361,400 .","In his new position , Dynes will earn $ 395,000 , a significant increase over Atkinson 's salary of $ 361,400 .",equivalent,2039
2,""" The company has always made , and continues to make , exceptional customer service and customer satisfaction a top priority , "" the statement said .","The Company has always made , and continues to make , exceptional customer service and customer satisfaction a top priority in all business practices , "" AOL added .",equivalent,3972
3,"CIA Director George Tenet said the two men were "" defined by dedication and courage . """,""" [ They ] were defined by dedication and courage , "" said the CIA 's director George Tenet .",equivalent,1346
4,"At his sentencing , Avants had tubes in his nose and a portable oxygen tank beside him .","Avants , wearing a light brown jumpsuit , had tubes in his nose and a portable oxygen tank beside him .",equivalent,530
5,"They were just making sure they dotted all the I 's and crossed all the T 's , "" said Schooff .","They were just making sure they dotted all the I 's and crossed all the T 's , "" said Blair Schoof , executive director of music for AOL Europe .",equivalent,1389
6,"One question was whether France , which infuriated Washington by leading the charge against U.N. authorization for the war , would vote "" Yes "" or abstain .","France , which infuriated Washington by leading the charge against U.N. approval for the war , also sought changes .",not_equivalent,415
7,Both have aired videotapes of ex-president Saddam Hussein encouraging Iraqis to fight the U.S. occupation of Iraq .,"Both have aired videotapes apparently from deposed Iraqi President Saddam Hussein , encouraging Iraqis to fight the U.S. occupation .",equivalent,155
8,He said Qantas would also cut capital spending by retiring some aircraft and deferring delivery of some new planes .,It would also further reduce capital expenditure by retiring some aircraft and delaying the delivery of new places .,equivalent,3614
9,The justices declined without comment to review the ruling that upheld physicians ' rights to speak freely with their patients .,The justices declined without comment to review a lower-court ruling that said doctors should be able to speak frankly with their patients .,equivalent,3315


In [12]:
#Define model check point and tokenizer
checkpoint="bert-base-cased"
tokenizer= AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [13]:
tokenizer("Hello, first sentence", "Hello second sentence")

{'input_ids': [101, 8667, 117, 1148, 5650, 102, 8667, 1248, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:

def tokenize_function(example):
  outputs = tokenizer(example['sentence1'], example['sentence2'],truncation=True, padding="max_length",max_length=128)
  return outputs

In [17]:
tokenize_function(raw_datasets['train'][:5])

{'input_ids': [[101, 7277, 2180, 5303, 4806, 1117, 1711, 117, 2292, 1119, 1270, 107, 1103, 7737, 107, 117, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 11336, 6732, 3384, 1106, 1140, 1112, 1178, 107, 1103, 7737, 107, 117, 7277, 2180, 5303, 4806, 1117, 1711, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10684, 2599, 9717, 1161, 2205, 11288, 1377, 112, 188, 1196, 4147, 1103, 4129, 1106, 19770, 2787, 1107, 1772, 1111, 109, 123, 119, 126, 3775, 119, 102, 10684, 2599, 9717, 1161, 3306, 11288, 1377, 112, 188, 1107, 1876, 1111, 109, 5691, 1495, 1550, 1105, 1962, 1122, 1106, 19770, 2787, 1111, 109, 122, 119, 129, 3775, 1107, 1772, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
## Tokenize the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True,remove_columns=["idx","sentence1","sentence2"] )

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [30]:
tokenized_datasets = tokenized_datasets.rename_column("label","labels")

In [31]:
tokenized_datasets["train"].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [32]:
tokenized_datasets.set_format("torch")

A look at the model using sequence classification head


In [33]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
##Define Dataloaders to convert data elements into batches and iterate over them

In [34]:
from torch.utils.data import DataLoader
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
  train_dataloader = DataLoader(
      tokenized_datasets['train'],
      shuffle=True,
      batch_size=train_batch_size,
      #collate_fn= data_collator
  )
  eval_dataloader = DataLoader(
      tokenized_datasets['validation'],
      shuffle= False,
      batch_size= eval_batch_size,
      #collate_fn= data_collator
  ) 
  return train_dataloader, eval_dataloader
train_dataloader, eval_dataloader= create_dataloaders()
#Let's have a look at our train and evaluation dataloaders to check a batch can go through the model.
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}


{'attention_mask': torch.Size([8, 128]),
 'input_ids': torch.Size([8, 128]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 128])}

In [35]:
outputs=model(**batch)
outputs

SequenceClassifierOutput([('loss', tensor(0.8441, grad_fn=<NllLossBackward>)),
                          ('logits', tensor([[ 0.1168, -0.5691],
                                   [ 0.1555, -0.5720],
                                   [ 0.0975, -0.5846],
                                   [ 0.1444, -0.5675],
                                   [ 0.1237, -0.5884],
                                   [ 0.1575, -0.5718],
                                   [ 0.1188, -0.5677],
                                   [ 0.1466, -0.5609]], grad_fn=<AddmmBackward>))])

et's have a look at our train and evaluation dataloaders to check a batch can go through the model.

In [36]:
##provide metric for model evaluation

metric= load_metric("glue","mrpc")

In [ ]:
##use this object on some predictions by calling the compute function

In [39]:
logits= outputs.logits
print(logits)

tensor([[ 0.1168, -0.5691],
        [ 0.1555, -0.5720],
        [ 0.0975, -0.5846],
        [ 0.1444, -0.5675],
        [ 0.1237, -0.5884],
        [ 0.1575, -0.5718],
        [ 0.1188, -0.5677],
        [ 0.1466, -0.5609]], grad_fn=<AddmmBackward>)


In [41]:
predictions = torch.argmax(logits, dim=-1)

In [42]:
metric.compute(predictions=predictions, references=batch["labels"])

{'accuracy': 0.375, 'f1': 0.0}

# FINE TUNING 

In [43]:
###Fine Tuning our model
hyperparameters={
    "learning_rate":3e-5,
    "num_epochs":3,
    "train_batch_size": 8, #Actual batch size will be this x 8
    "eval_batch_size": 32,#Actual batch size will be this x 8
    "seed":42,
}

In [45]:
from transformers import AdamW

In [ ]:
import accelerate
def training_function(model):
   # Initialize accelerator
    accelerator = Accelerator()
   # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    train_dataloader,eval_dataloader= create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"]
    )
    #set the seed befor model instantiation as it determines the random head
    set_seed(hyperparameters["seed"])
    #instantiate the optimizer
    optimizer = AdamW(params=model.parameters(), lr= hyperparameters["learning_rate"])
    #unpack the objects in the same order we gave them to the
    # prepare method.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader,eval_dataloader)
    num_epochs=hyperparameters["num_epochs"]

   # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler= get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader)*num_epochs,

    )

    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not acclerator.is_main_process)
    # Now we train the model
    for epoch in range(num_epochs):
      model.train()
      for step, batch in enumerate(train_dataloader):
        outputs=model(**batch)
        loss=outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

      model.eval()
      all_predictions=[]
      all_labels=[]

      for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
          outputs= model(**batch)
        predictions = outputs.logits.argmax(dim=-1)

        # We gather predictions and labels from the 8 TPUs to have them all.

        all_predictions.append(accelerator.gather(predictions))
        all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.

        all_predictions = torch.cat(all_predictions)[:len(tokenized_datasets["validation"])]
        all_labels = torch.cat(all_labels)[:len(tokenized_datasets["validation"])]

        eval_metric = metric.compute(predictions= all_predictions, reference=all_labels)
        
        # Use accelerator.print to print only on the main process.

        accelerator.print(f"epoch {epoch}:", eval_metric)
    
        


SyntaxError: ignored

In [ ]:
#Launch

from accelerate import notebook_launcher
notebook_launcher(training_function, (model,))